In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os

import torchtext
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import glob

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!git clone 'https://github.com/ThanhChinhBK/vietnews'

Cloning into 'vietnews'...
remote: Enumerating objects: 143827, done.
remote: Counting objects: 100% (143827/143827), done.
remote: Compressing objects: 100% (143815/143815), done.
remote: Total 143827 (delta 11), reused 143827 (delta 11), pack-reused 0
Receiving objects: 100% (143827/143827), 194.68 MiB | 18.99 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Checking out files: 100% (150704/150704), done.


In [ ]:
SOS_token = 0
EOS_token = 1

def flatten_list(lists):
  flat_list = [item for sublist in lists for item in sublist]
  return flat_list

def build_dict(lists):
  word2index = {}
  word2count = {}
  index2word = {0: "SOS", 1: "EOS"}
  n_words = 2  # Count SOS and EOS

  flatten = ''.join(flatten_list(lists))
  for word in flatten.split(' '):
    if word not in word2index:
      word2index[word] = n_words
      word2count[word] = 1
      index2word[n_words] = word
      n_words += 1
    else:
      word2count[word] += 1
  
  return word2index, word2count, index2word, n_words


In [ ]:
def readLangs(directory):
    files_path = glob.glob(f'{directory}/*')
    print("Reading lines...")
    sample = []
    text = []
    target = []
    for file in os.listdir(directory):
      with open(os.path.join(directory,file), 'r') as f:
          file_content = f.readlines()
          abstract = file_content[2]
          body = ' '.join(file_content[3:]).replace('\n', '').replace('.\n', '').rstrip("\n")
          pairs = [abstract, body]
          sample.append(pairs)
          text.append(body)
          target.append(abstract)

    return text, target, sample

In [ ]:
def prepareData(directory):
    input_lang, output_lang, pairs = readLangs(directory)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    word2index, word2count, index2word, n_words = build_dict(input_lang)
    print("Counted words:")
    print(n_words)

    return input_lang, output_lang, pairs, word2index, word2count, index2word, n_words


input_lang, output_lang, pairs, word2index, word2count, index2word, n_words = prepareData('/content/vietnews/data/train_tokenized')
print(random.choice(pairs))

Reading lines...
Read 105418 sentence pairs
Counting words...
Counted words:
309491
['Cơ_quan điều_tra xác_định ông Sự đã lợi_dụng chức_vụ quyền_hạn trong việc sử_dụng nguồn tiền của Vinashin gửi vào OceanBank để một_số cá_nhân thuộc Vinashin nhận , chiếm_đoạt hơn 105 tỉ đồng .\n', ' Ngày 26-1 , thông_tin từ Bộ Công_an cho_biết Cơ_quan Cảnh_sát điều_tra Bộ Công_an ( C 46 ) đã ra quyết_định khởi_tố vụ án , khởi_tố bị_can , lệnh bắt tạm giam ông Nguyễn_Ngọc_Sự , nguyên chủ_tịch hội_đồng thành_viên Tập_đoàn Công_nghiệp tàu_thuỷ Việt_Nam_Vinashin ( SBIC ) , để điều_tra về tội lợi_dụng chức_vụ quyền_hạn trong khi thi_hành công_vụ . Đồng_thời Cơ_quan điều_tra thi_hành lệnh khám_xét nhà ông Sự để phục_vụ công_tác điều_tra . Các quyết_định và lệnh trên đã được Viện_kiểm_sát nhân_dân tối_cao phê_chuẩn . Ông Sự bị bắt trong quá_trình C46 điều_tra giai_đoạn II vụ án Hà_Văn_Thắm và đồng_phạm . Quá_trình điều_tra , tài_liệu chứng_cứ thu_thập được xác_định Nguyễn_Ngọc_Sự , đã có hành_vi lợi_dụng chứ

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=2000):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [word2index.get(word,10) for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=2000):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=2000):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

In [ ]:
def evaluateInput(input_sentence):
    output_words, attentions = evaluate(encoder1, attn_decoder1, input_sentence)
    print(outputs_words)